In [1]:
#Importing OpenCV Library for basic image processing functions
import cv2
# Numpy for array related functions
import numpy as np
# Dlib for deep learning based Modules and face landmark detection
import dlib
#face_utils for basic operations of conversion
from imutils import face_utils
#importing Tkinter for registration
from tkinter import *
from tkinter import messagebox

In [2]:
import os
os.chdir('C:\\Users\\STUTI DUTTA\\Desktop\\amity\\Machine Learning\\Project\\Drowsiness Detection\\Details')
os.getcwd()

'C:\\Users\\STUTI DUTTA\\Desktop\\amity\\Machine Learning\\Project\\Drowsiness Detection\\Details'

In [3]:
screen=Tk()
screen.title("Sign In")
screen.geometry("500x300")
#screen.resizable(False,False)
def submit_value():
    name_info=name_value.get()
    phone_info=phone_value.get()
    address_info=address_value.get()
    city_info=city_value.get()
    state_info=state_value.get()
    gender_info=gender_value.get()
    
    if (name_info==""):
        messagebox.showerror("Enter your name")
    elif (phone_info==""):
        messagebox.showerror("Enter your contact no.")
    elif (address_info==""):
        messagebox.showerror("Enter your address")
    elif (city_info==""):
        messagebox.showerror("Enter your city")
    elif (state_info==""):
        messagebox.showerror("Enter your city")
    elif(gender_info==""):
        messagebox.showerror("Enter your gender")
    else:
        Label(screen, text="Submitted Successfully!", font="Ariel 10 bold", fg="red" ).grid(row=9, column=3)
    with open(name_info+".txt", 'w') as f:
        f.write(name_info+"\n")
        f.write(phone_info+"\n")
        f.write(address_info+"\n")
        f.write(state_info+"\n")
        f.write(gender_info+"\n")
    

def clear_value():
    name_box.delete(0,END)
    phone_box.delete(0,END)
    address_box.delete(0,END)
    city_box.delete(0,END)
    state_box.delete(0,END)
    gender_box.delete(0,END)
    
    

Label(screen,text="Registration Form", font="Ariel 20 bold underline").grid(row=0,column=3)

#details to be filled
name=Label(screen,text="Name")
name.grid(row=1,column=2)

phone=Label(screen,text="Contact No.")
phone.grid(row=2,column=2)

address=Label(screen,text="Address")
address.grid(row=3,column=2)

city=Label(screen,text="City")
city.grid(row=4,column=2)

state=Label(screen,text="State")
state.grid(row=5,column=2)

gender=Label(screen,text="Gender")
gender.grid(row=6,column=2)

#what type of input

name_value=StringVar()
phone_value=StringVar()
address_value=StringVar()
city_value=StringVar()
state_value=StringVar()
gender_value=StringVar()
check_value=IntVar()

#For input

name_box=Entry(screen, textvariable=name_value)
name_box.grid(row=1, column=3)


phone_box=Entry(screen, textvariable=phone_value)
phone_box.grid(row=2, column=3)

address_box=Entry(screen, textvariable=address_value)
address_box.grid(row=3, column=3)

city_box=Entry(screen, textvariable=city_value)
city_box.grid(row=4, column=3)

state_box=Entry(screen, textvariable=state_value)
state_box.grid(row=5, column=3)

gender_box=Entry(screen, textvariable=gender_value)
gender_box.grid(row=6, column=3)

check_box=Checkbutton(text="Remember Me!", variable=check_value)
check_box.grid(row=7, column=3)

Button(text="Submit", command=submit_value).grid(row=8, column=3)
Button(text="Clear").grid(row=10, column=4)

mainloop()

In [4]:
os.chdir('C:\\Users\\STUTI DUTTA\\Desktop\\amity\\Machine Learning\\Project\\Drowsiness Detection')
os.getcwd()

'C:\\Users\\STUTI DUTTA\\Desktop\\amity\\Machine Learning\\Project\\Drowsiness Detection'

In [5]:
#Initializing the camera and taking the instance
cam = cv2.VideoCapture(0)

In [6]:
#Initializing the face detector and landmark detector
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [7]:
#status marking for current state
sleep = 0
drowsy = 0
active = 0
status=""
color=(0,0,0)

In [8]:
def compute(ptA,ptB):
    dist = np.linalg.norm(ptA - ptB)
    return dist


In [9]:
def blinked(a,b,c,d,e,f):
    up = compute(b,d) + compute(c,e)
    down = compute(a,f)
    ratio = up/(2.0*down)
    
    #Checking if it is blinked
    if(ratio>0.25):
        return 2
    elif(ratio>0.21 and ratio<=0.25):
        return 1
    else:
        return 0


In [10]:
os.chdir('C:\\Users\\STUTI DUTTA\\Desktop\\amity\\Machine Learning\\Project\\Drowsiness Detection\\History')
os.getcwd()

'C:\\Users\\STUTI DUTTA\\Desktop\\amity\\Machine Learning\\Project\\Drowsiness Detection\\History'

In [11]:
img=0
while True:
    _, frame = cam.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = detector(gray)
    #detected face in faces array
    for face in faces:
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.bottom()

        face_frame = frame.copy()
        cv2.rectangle(face_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        landmarks = predictor(gray, face)
        landmarks = face_utils.shape_to_np(landmarks)

        #The numbers are actually the landmarks which will show eye
        left_blink = blinked(landmarks[36],landmarks[37], 
            landmarks[38], landmarks[41], landmarks[40], landmarks[39])
        right_blink = blinked(landmarks[42],landmarks[43], 
            landmarks[44], landmarks[47], landmarks[46], landmarks[45])
        
        #Now judge what to do for the eye blinks
        if(left_blink==0 or right_blink==0):
            sleep+=1
            drowsy=0
            active=0
            if(sleep>6):
                status="SLEEPING !!!"
                color = (255,0,0)

        elif(left_blink==1 or right_blink==1):
            sleep=0
            active=0
            drowsy+=1
            if(drowsy>6):
                status="Drowsy !"
                color = (0,0,255)

        else:
            drowsy=0
            sleep=0
            active+=1
            if(active>6):
                status="Active :)"
                color = (0,255,0)
                
        cv2.putText(frame, status, (100,100), cv2.FONT_HERSHEY_SIMPLEX, 1.2, color,3)

        for n in range(0, 68):
            (x,y) = landmarks[n]
            cv2.circle(face_frame, (x, y), 1, (255, 255, 255), -1)
    face_frame = frame.copy()
    #cv2.imshow("Frame", frame)
    cv2.imshow("Result of detector", face_frame)
    k = cv2.waitKey(1)
    #k=cv2.waitKey(1)
    if k%256==27:
        print("Escape hit, closing")
        break
    elif k%256==32:
        img_name="opencv_frame_{}.jpeg".format(img)
        cv2.imwrite(img_name,frame)
        print("screenshot taken")
        img=img+1

cam.release()

cam.destroyAllWindows()


screenshot taken
screenshot taken
Escape hit, closing


AttributeError: 'cv2.VideoCapture' object has no attribute 'destroyAllWindows'